# Ex2

## Q1

TBD

## Q2

TBD

## Q3

TBD

## Q4

### Get Corpus and Dictionary

In [1]:
import urllib2
import zipfile
import os

data_folder = 'corpus_combined'
corpus_file = data_folder + '.zip'

if not os.path.exists(data_folder):
    os.makedirs(data_folder)
    
response = urllib2.urlopen('http://www2.mta.ac.il/~gideon/courses/nlp/data/' + corpus_file)
zipcontent = response.read()

with open(corpus_file, 'w') as f:
    f.write(zipcontent)
    
zip_ref = zipfile.ZipFile(corpus_file, 'r')
zip_ref.extractall(data_folder)
zip_ref.close()

os.remove(corpus_file)

response = urllib2.urlopen('http://www2.mta.ac.il/~gideon/courses/nlp/data/word_list_20k.txt')
dictionary = response.read().split('\n')

### Prepare training and test set

In [2]:
import urllib2

# training
a_tail_of_two_cities = urllib2.urlopen('http://www.gutenberg.org/files/98/98-0.txt').read().decode('utf8')
oliver_twist = urllib2.urlopen('http://www.gutenberg.org/ebooks/730.txt.utf-8').read().decode('utf8')
training = [a_tail_of_two_cities, oliver_twist]
# testing
persuasion = open(data_folder + '/persuasion.txt', 'r').read()
testing = [persuasion]

### Bigram

#### Pre-Processing

In [31]:
from chapterize import Book
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters

punkt_param = PunktParameters()
punkt_param.abbrev_types = set(['dr', 'vs', 'mr', 'mrs', 'prof', 'inc', 'ms'])

# for book in training:
# removing header, footer and chapters
# book = Book(book).getContent()
book = Book(a_tail_of_two_cities).getContent()
# lowercasing
book = book.lower()
# split to sentences
sentences = PunktSentenceTokenizer(punkt_param).tokenize(book)
ps = PorterStemmer()
book = []
for sentence in sentences:
    # tokenizing
    tokens = RegexpTokenizer(r'\w+').tokenize(sentence)
    # removing stopwords
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    # stemming
    tokens = [ps.stem(token) for token in tokens]
    book.append(tokens)

In [37]:
from nltk import bigrams, trigrams
from collections import defaultdict

model = defaultdict(lambda: defaultdict(lambda: 0))

for sentence in book:
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        model[(w1, w2)][w3] += 1

for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

In [39]:
print(model['best', 'time']['worst'])

0.5
